In [1]:
#Importing Python packages
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [2]:
#Importing ACSdata

ACSdata = pd.read_csv('data/ACSdata.csv', sep=',', header=0, skipinitialspace=True)

In [3]:
ACSdata.shape

(3190040, 46)

In [4]:
#EXPLORING THE DATA

#Generating summary statistics - age
ACSdata['AGEP'].describe().astype('int64')

#Age range (0-96), data set needs to be filtered to include persons of marriageable age(> 18)

count    3190040
mean          41
std           23
min            0
25%           21
50%           42
75%           60
max           96
Name: AGEP, dtype: int64

In [5]:
#Rows in which value of AGEP column in  less than 18
is_lt18 = ACSdata.apply(lambda x: True if x['AGEP'] < 18 else False, axis=1)

#Count number of True in the series
numofRows = len(is_lt18[is_lt18 ==True].index)

print('Number of Rows in ACSdata in which Age < 18:', numofRows)

Number of Rows in ACSdata in which Age < 18: 659314


In [6]:
#Wrangling - Removing persons from the dataset who are < 18 years and creating a new df (ACSmarry)
ACSmarry = ACSdata[ACSdata.AGEP >= 18]
print(ACSmarry.shape)

(2530726, 46)


In [7]:
#EXPLORING THE DATAFRAME (ACSmarry)

#Exploring the marriage indicator
#Generating frequency table - marriage
ACSmarry['MAR'].value_counts()

1    1360300
5     671044
3     285664
2     169374
4      44344
Name: MAR, dtype: int64

In [8]:
ACSmarry['MAR'].value_counts(sort=True) #Marital status

1    1360300
5     671044
3     285664
2     169374
4      44344
Name: MAR, dtype: int64

In [9]:
ACSmarry['MAR'].value_counts(sort=True, normalize=True) * 100 #Marital status

1    53.751374
5    26.515869
3    11.287828
2     6.692704
4     1.752224
Name: MAR, dtype: float64

In [10]:
#Wrangling - Removing persons who are divorced, seperated or widowed.


# delete all rows with column 'Age' has value 30 to 40 
indexNames = ACSmarry[(ACSmarry['MAR'] >= 2) & (ACSmarry['MAR'] <= 4) ].index #The condition
ACSmarry.drop(indexNames , inplace=True) #Dropping rows that meet the condition

C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
#Printing ACSmarry to csv

#fields = ['AGEP','MAR','SERIALNO','SPORDER']
#Create a sample of top 100 to explore a smaller file size
#ACSsample = ACSmarry.head(10000)
# update to the path on your local drive
#ACSsample.to_csv(r'testdata/sample1.csv', columns=fields)

In [12]:
ACSmarry['MAR'].value_counts(sort=True) #Marital status

1    1360300
5     671044
Name: MAR, dtype: int64

In [13]:
#Transformaing--> Marrital Status (yes/no)
ACSmarry.loc[ACSmarry.MAR == 5, 'MARRIED'] = 0
ACSmarry.loc[ACSmarry.MAR == 1, 'MARRIED'] = 1

ACSmarry['MARRIED'].value_counts(sort=True) 

C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1.0    1360300
0.0     671044
Name: MARRIED, dtype: int64

In [14]:
#Using ADJINC to calculate wages/earning and income variables
#Note--> this code will need to be adjusted for the multi-year file

ACSmarry['WAGES'] = ACSmarry.WAGP * (ACSmarry.ADJINC / 1000000 )
ACSmarry['INCOME'] = ACSmarry.PINCP * (ACSmarry.ADJINC / 1000000 )
ACSmarry['EARN'] = ACSmarry.PERNP * (ACSmarry.ADJINC / 1000000 )

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [15]:
ACSmoney = ACSmarry[['WAGES', 'WAGP', 'INCOME', 'PINCP', 'EARN', 'PERNP']]

ACSmoney.describe().astype('int64')

,WAGES,WAGP,INCOME,PINCP,EARN,PERNP
count,2031344,2031344,2031344,2031344,2031344,2031344
mean,33547,33176,44435,43943,35767,35371
std,57900,57259,65381,64658,60919,60245
min,0,0,-9201,-9100,-9100,-9000
25%,0,0,9100,9000,0,0
50%,14156,14000,26998,26700,17190,17000
75%,46514,46000,55615,55000,49548,49000
max,744235,736000,1580488,1563000,1274098,1260000


In [16]:
#print(ACSmarry[['ADJINC', 'WAGP', 'WAGES', 'PINCP', 'INCOME', 'PERNP', 'EARN']].head())

In [17]:
#Running summary statistics for continuous variables
ACScont = ACSmarry[['WAGES', 'INCOME', 'EARN', 'AGEP', 'JWMNP', 'WKHP']]

ACScont.describe().astype('int64')

,WAGES,INCOME,EARN,AGEP,JWMNP,WKHP
count,2031344,2031344,2031344,2031344,1162544,1387664
mean,33547,44435,35767,46,27,38
std,57900,65381,60919,18,23,13
min,0,-9201,-9100,18,1,1
25%,0,9100,0,31,12,35
50%,14156,26998,17190,46,20,40
75%,46514,55615,49548,61,35,45
max,744235,1580488,1274098,96,160,99


In [18]:
#Frequency tables and possible transformation of categorical variables

#Citizenship status

ACSmarry['CIT'].value_counts(sort=True) 

1    1701886
4     158692
5     140083
3      19726
2      10957
Name: CIT, dtype: int64

In [19]:
#Transformaing--> Citizen (yes/no)
ACSmarry.loc[ACSmarry.CIT == 5, 'CITIZEN'] = 0
ACSmarry.loc[ACSmarry.CIT != 5, 'CITIZEN'] = 1

ACSmarry['CITIZEN'].value_counts(sort=True) 

1.0    1891261
0.0     140083
Name: CITIZEN, dtype: int64

In [20]:
#Transforming number of times married
ACSmarry['MARHT'].value_counts(sort=True) #Number of times married

1.0    1031074
2.0     257976
3.0      71250
Name: MARHT, dtype: int64

In [21]:
ACSmarry['MARHT'].value_counts(sort=True, normalize=True) * 100 

1.0    75.797545
2.0    18.964640
3.0     5.237815
Name: MARHT, dtype: float64

In [22]:
#Marital status by the number of times married
pd.crosstab(ACSmarry.MAR, ACSmarry.MARHT, margins=True)

MARHT,1.0,2.0,3.0,All
MAR,,,,
1,1031074,257976,71250,1360300
All,1031074,257976,71250,1360300


In [23]:
#Creating variable: Previously married (# of times)
ACSmarry.loc[ACSmarry.MAR == 5, 'MAR_TIMES'] = 0 #Married zero times
ACSmarry.loc[(ACSmarry.MAR != 5) & (ACSmarry.MARHT == 1), 'MAR_TIMES'] = 1 #married 1 time
ACSmarry.loc[(ACSmarry.MAR != 5) & (ACSmarry.MARHT != 1), 'MAR_TIMES'] = 2 #married more than 1 times

ACSmarry['MAR_TIMES'].value_counts(sort=True) 

1.0    1031074
0.0     671044
2.0     329226
Name: MAR_TIMES, dtype: int64

In [24]:
#Mobility:  (lived here 1 year aga)
ACSmarry['MIG'].value_counts(sort=True)

1.0    1752560
3.0     265839
2.0      12945
Name: MIG, dtype: int64

In [25]:
#Transforming MIG to "Mover, or changed locations 1 year or less ago"
ACSmarry.loc[ACSmarry.MIG != 1, 'MOVER'] = 1 #Moved, changed location
ACSmarry.loc[ACSmarry.MIG == 1, 'MOVER'] = 0 #Did not move or change location

ACSmarry['MOVER'].value_counts(sort=True)

0.0    1752560
1.0     278784
Name: MOVER, dtype: int64

In [26]:
ACSmarry['MOVER'].value_counts(sort=True, normalize=True) * 100

0.0    86.275884
1.0    13.724116
Name: MOVER, dtype: float64

In [27]:
#Transforming educational attainment variable (SCHL)

ACSmarry['SCHL'].value_counts(sort=True)

16.0    456198
21.0    407781
19.0    313244
22.0    179471
20.0    167012
18.0    145340
17.0     74740
14.0     44967
23.0     44852
15.0     38093
24.0     29724
13.0     29327
1.0      26821
12.0     22642
11.0     17753
9.0      14961
10.0      5504
8.0       3785
6.0       3422
7.0       2614
5.0       1569
4.0        714
3.0        413
2.0        397
Name: SCHL, dtype: int64

In [28]:
ACSmarry.loc[ACSmarry.SCHL < 16, 'EDUCATION'] = 'NDIP' #No high school diploma or GED
ACSmarry.loc[(ACSmarry.SCHL == 16) | (ACSmarry.SCHL == 17), 'EDUCATION'] = 'HS' #HS diploma or GED
ACSmarry.loc[(ACSmarry.SCHL == 18) | (ACSmarry.SCHL == 19) | (ACSmarry.SCHL == 20), 'EDUCATION'] = AA_DEG #less thhan a college degree or an associate's
ACSmarry.loc[(ACSmarry.SCHL == 21), 'EDUCATION'] = BA_DEG #Bachelor's degree
ACSmarry.loc[(ACSmarry.SCHL == 22) | (ACSmarry.SCHL == 23) | (ACSmarry.SCHL == 24), 'EDUCATION'] = GRAD #Graduate degree

ACSmarry['EDUCATION'].value_counts(sort=True)

2.0    625596
1.0    530938
3.0    407781
4.0    254047
0.0    212982
Name: EDUCATION, dtype: int64

In [29]:
#Marital status by educational attainment
pd.crosstab(ACSmarry.MAR, ACSmarry.EDUCATION, margins=True)

EDUCATION,0.0,1.0,2.0,3.0,4.0,All
MAR,,,,,,
1,126647,339053,390516,296056,208028,1360300
5,86335,191885,235080,111725,46019,671044
All,212982,530938,625596,407781,254047,2031344


In [ ]:
#JWTR: Means of Transportation to work
# 0 for tranportation that dont let you socialize:Car, truck, Van, Motorcyclle, Bicycle, Work at home and Other
# 1 for JWTR that let you socialize: Bus, trolley car, Subway, Railroad, Ferryboat, Taxicab, Walked

ACSmarry['JWTR_T'] = ACSmarry['JWTR']
ACSmarry.loc[(ACSmarry.JWTR_T ==1)|(ACSmarry.JWTR_T ==8)|(ACSmarry.JWTR_T ==9)|(ACSmarry.JWTR_T ==11)|(ACSmarry.JWTR_T ==12),'JWTR_T'] = 0
ACSmarry.loc[(ACSmarry.JWTR_T ==2)|(ACSmarry.JWTR_T ==3)|(ACSmarry.JWTR_T ==4)|(ACSmarry.JWTR_T ==5)|(ACSmarry.JWTR_T ==6)|(ACSmarry.JWTR_T ==7)|(ACSmarry.JWTR_T ==10),'JWTR_T'] = 1

In [ ]:
#After Transformation
ACSmarry['JWTR_T'].value_counts(sort=True)

In [ ]:
#Before Transformation
ACSmarry['JWTR'].value_counts(sort=True)

In [ ]:
#SEX: Transforming to 0/1
# Female = 0; Male =1
ACSmarry['SEX_T'] = ACSmarry['SEX']
ACSmarry.loc[(ACSmarry.SEX_T ==2),'SEX_T'] = 0

In [ ]:
#After Transformation
ACSmarry['SEX_T'].value_counts(sort=True)

In [ ]:
#Before Transformation
ACSmarry['SEX_T'].value_counts(sort=True)

In [ ]:
#Using ESR to created Employed/Unemployed
# Employed: 1,2,4,5 and unemployed 3,6
ACSmarry['ESR_T'] = ACSmarry['ESR']
ACSmarry.loc[(ACSmarry.ESR_T == 1)|(ACSmarry.ESR_T == 2)|(ACSmarry.ESR_T == 4)|(ACSmarry.ESR_T ==5),'EST_T'] = 1
ACSmarry.loc[(ACSmarry.ESR_T == 3)|(ACSmarry.ESR_T ==6),'ESR_T'] = 0

In [ ]:
#After Transformation
ACSmarry['ESR_T'].value_counts(sort=True)

In [ ]:
#Before Transformation
ACSmarry['ESR_T'].value_counts(sort=True)

In [ ]:
#Nativity (Transformaing to 0/1)
# Foreign = 0; Native = 1
ACSmarry['NATIVITY_T'] = ACSmarry['NATIVITY']
ACSmarry.loc[(ACSmarry.NATIVITY_T == 2),'NATIVITY_T'] = 0

In [ ]:
#After Transformation
ACSmarry['NATIVITY_T'].value_counts(sort=True)

In [ ]:
#Before Transformation
ACSmarry['NATIVITY'].value_counts(sort=True)

In [ ]:
# Job Code Transformation function
#input must be interger data type 

def jobid_to_jobclass(occp):
    try: 
        occp = int(occp)
    except:
        return occp
    else:
        if occp <= 430:
            jobclass = 'MGR'
        elif occp <= 740:
            jobclass = 'BUS'
        elif occp <= 950:
            jobclass = 'FIN'
        elif occp <= 1240:
            jobclass = 'CMM'
        elif occp <= 1560:
            jobclass = 'ENG'
        elif occp <= 1965:
            jobclass = 'SCI'
        elif occp <= 2060:
            jobclass = 'CMS'
        elif occp <= 2160:
            jobclass = 'LGL'
        elif occp <= 2550:
            jobclass = 'EDU'
        elif occp <= 2920:
            jobclass = 'ENT'
        elif occp <= 1240:
            jobclass = 'CMM'
        elif occp <= 3540:
            jobclass = 'MED'
        elif occp <= 3655:
            jobclass = 'HLS'
        elif occp <= 3955:
            jobclass = 'PRT'
        elif occp <= 4150:
            jobclass = 'EAT'
        elif occp <= 4250:
            jobclass = 'CMM'
        elif occp <= 4650:
            jobclass = 'PRS'
        elif occp <= 4965:
            jobclass = 'SAL'
        elif occp <= 5940:
            jobclass = 'OFF'
        elif occp <= 6130:
            jobclass = 'FFF'
        elif occp <= 6765:
            jobclass = 'CON'
        elif occp <= 6940:
            jobclass = 'EXT'
        elif occp <= 7630:
            jobclass = 'RPR'
        elif occp <= 8965:
            jobclass = 'PRD'
        elif occp <= 9750:
            jobclass = 'TRN'
        elif occp <= 9830:
            jobclass = 'MIL'
        elif occp <= 9920:
            jobclass = 'UNE' 
        else:
            jobclass = 'ERROR'  
    return jobclass

In [ ]:
print(jobid_to_jobclass(9000))

In [ ]:
# Transformation for Occupation Code OCCP from jobid to occupation code (ex: 10 to MRG)
OCCP_T = []
for i in ACSmarry['OCCP']:
    jobclass = str(jobid_to_jobclass(i))
    OCCP_T.append(jobclass)
ACSmarry['OCCP_T']= pd.DataFrame(OCCP_T)

In [ ]:
#Next two lines is to check if everything is copied from OCCP to OCCP_T
len(ACSmarry['OCCP_T'])

In [ ]:
len(ACSmarry['OCCP'])

In [ ]:
print(OCCP_T)

In [ ]:
#Function to normalize PUMA length (making sure that PUMA = len(5), adding leading zeroes where missing)
def normalize_PUMA(puma):
    try: 
        test = int(puma)
        puma_t = str(test)
    except:
        return prin('Error: not an integer')
    else:
        if len(puma_t) == 3:
            puma_t = '00' + str(puma_t)
        elif len(puma_t)== 4:
            puma_t = '0' + str(puma_t)
        elif len(puma_t)== 5:
            puma_t = puma_t
        else:
            puma_t = 'size error'
        return puma_t

In [ ]:
#Create a new column to store the normalized Puma from say 100 to 00100
PUMA_T = []
for p in ACSdata['PUMA']:
    new_p = normalize_PUMA(p)
    PUMA_T.append(new_p)
ACSdata['PUMA_T'] = pd.DataFrame(PUMA_T)

In [ ]:
#Print length of the transformed variable
print(len(PUMA_T))

In [ ]:
#Print lenght of the new PUMA_T
print(len(ACSdata['PUMA_T']))

In [ ]:
#Print PUMA_T
print(PUMA_T)

In [ ]:
#Normalize the length ST due to possible issues with the leading zero
ST_T = []
ST_T =[ST if len(ACSdata['ST']) == 2 else '0'+str(ST) for ST in ACSdata['ST'] ]
ACSdata['ST_T'] = pd.DataFrame(ST_T)

In [ ]:
#print ST_T
print(ST_T)

In [30]:
#Condensing the race categories

#ACSmarry.loc[ACSmarry.RACP1 == 1, 'RACE'] = 1 #White
#ACSmarry.loc[(ACSmarry.RACP1 == 2) 'RACE'] = 2 #Black, African American
#ACSmarry.loc[(ACSmarry.RACP1 == 3) | (ACSmarry.RACP1 == 4) | (ACSmarry.RACP1 ==5), 'RACE'] = 3 #American Indian/Alaskan Native
#ACSmarry.loc[(ACSmarry.RACP1 == 6) 'RACE'] = 4 #Asian
#ACSmarry.loc[(ACSmarry.RACP1 == 7) 'RACE'] = 5 #Native Hawaiian/Pacific Islander
#ACSmarry.loc[(ACSmarry.RACP1 == 8) 'RACE'] = 2 #Black, African American
#ACSmarry.loc[(ACSmarry.SCHL == 18) | (ACSmarry.SCHL == 19) | (ACSmarry.SCHL == 20), 'EDUCATION'] = 2 #less thhan a college degree or an associate's
#ACSmarry.loc[(ACSmarry.SCHL == 21), 'EDUCATION'] = 3 #Bachelor's degree
#ACSmarry.loc[(ACSmarry.SCHL == 22) | (ACSmarry.SCHL == 23) | (ACSmarry.SCHL == 24), 'EDUCATION'] = 4 #Graduate degree

#ACSmarry['RACE'] = ACSmarry['RACP1']

#Setting the condition to create American Indian/Alaskan
#conds = 

#Setting the value for the aforemnetioned condition
#codes = [3]


# Use np.select with a default of 3 (your "else" value)    
#df['progress'] = np.select(conds, choices, default = 3)

In [34]:
#Exporting ACSmarry to csv
ACSmarry.to_csv(r'data/ACSmarry.csv')
